Julia code for Kitaev model.
\section{Kitaev model without symmetry}

In [143]:
function index(N1::Int, N2::Int, c::Int, r::Int, atom::Int)
    #Calculate the index of the atoms in the (r,c) unit cell
    #A-sublattice atom = 0; B-subkattice: atom = 1
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c
    ind = 2 * n + 1 + atom
    return ind
end

function bits(tag::Int,i::Int, N1::Int, N2::Int)
    # Chenck the i-th element of the binary representation of tag
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
end
    
function StateConfig(tag::Int, N1::Int, N2::Int)
    #=The configuration of the state with tag.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers       
    Outputs: binary form of tag. type: 1D array=#
    
    dim = 2^(N1 * N2 * 2)
    if tag >= dim
        return println("Error: The tag is out of range.")
    else
        b = string(tag, base = 2, pad = N1 * N2 * 2)
        return b
    end
end

function flip(tag::Int, i::Int, j::Int, N1::Int, N2::Int)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
end

function KitaevRhom(J::Array{Float64,1}, N1::Int, N2::Int)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice
    Inputes: N1, N2: column and row numbers
             J: coupling constants
    Outpus: The Non-zero Hamiltonian element H[i] is in the 
            position (row[i], col[i])
            H, row, col : type: list =#
    SiteNum = N1 * N2 * 2
    dim = 2^SiteNum
    
    numNZero = 0  #count the number of non-zero element
    data = Float64[]
    row = Int[]
    col = Int[]
    for tag = 0: dim-1
        #println("The state numnber is: " , tag)
        temp = Dict{Int, Float64}() # A temporay dict
        
        for r = 0: N2-1
            for c = 0: N1-1
                ind = index(N1, N2, c, r, 0)
                # Sx Sx
                next = index(N1, N2, c, r, 1)
                
                ntag = flip(tag, ind, next, N1, N2)
                if ntag in keys(temp)
                    temp[ntag] += -J[1] /4
                else
                    temp[ntag] = -J[1] /4
                end  
                
                # Sy Sy
                next = index(N1, N2, c - 1, r, 1)
                
                ntag = flip(tag, ind, next, N1, N2) 
                if ntag in keys(temp)
                    temp[ntag] += J[2] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                else
                    temp[ntag] = J[2] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                end
                        
                #Sz Sz
                next = index(N1, N2, c, r - 1, 1)
                    
                if tag in keys(temp)
                    temp[tag] += - J[3] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                else
                    temp[tag] = - J[3] * (bits(tag,ind, N1, N2) - 0.5)* (bits(tag,next, N1, N2) - 0.5)
                end
            end
        end
                        
        odTemp = sort(collect(keys(temp)))
        for sf in odTemp
            append!(col, [tag + 1])
            append!(row, [sf + 1])
            append!(data,[temp[sf]])
            numNZero += 1
        end
    end
    
    #println("non zero elemens number =" , numNZero)
    #println("Percentage =" , numNZero/(dim * dim) * 100,"%")
    return col, row, data
end

function makeH(col, row, data)
    H = sparse(row, col, data)
    return H
end


makeH (generic function with 1 method)

In [147]:
using SparseArrays
using Arpack

@time begin
    N1 = 3
    N2 = 3
    J = [1.0,1.0,1.0]

    col, row, data = KitaevRhom(J, N1, N2)
    H =  makeH(col, row, data)
    
    e,x = eigs(H, which = :SR)
    println(e/(N1*N2*2))
end


[-0.198493, -0.198493, -0.198493, -0.195481, -0.195481, -0.194736]
  9.477507 seconds (17.57 M allocations: 2.170 GiB, 5.98% gc time)


In [148]:
using BenchmarkTools
N1 = 2
N2 = 3
J = [1.0,1.0,1.0]
@benchmark KitaevRhom(J, N1, N2)

BenchmarkTools.Trial: 
  memory estimate:  27.06 MiB
  allocs estimate:  200753
  --------------
  minimum time:     15.588 ms (10.38% GC)
  median time:      16.253 ms (10.43% GC)
  mean time:        16.819 ms (12.61% GC)
  maximum time:     21.749 ms (25.91% GC)
  --------------
  samples:          298
  evals/sample:     1

\section{Kitaev model with translation symmetry}

In [180]:
function index(N1::Int, N2::Int, c::Int, r::Int, atom::Int)
    #Calculate the index of the atoms in the (r,c) unit cell
    #A-sublattice atom = 0; B-subkattice: atom = 1
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c
    ind = 2 * n + 1 + atom
    return ind
end

function bits(tag::Int,i::Int, N1::Int, N2::Int)
    # Chenck the i-th element of the binary representation of tag
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
end

function StateConfig(tag::Int, N1::Int, N2::Int)
    #=The configuration of the state with tag.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers       
    Outputs: binary form of tag. type: 1D array=#
    
    dim = 2^(N1 * N2 * 2)
    if tag >= dim
        return println("Error: The tag is out of range.")
    else
        b = string(tag, base = 2, pad = N1 * N2 * 2)
        return b
    end
end

function flip(tag::Int, i::Int, j::Int, N1::Int, N2::Int)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
end

function cyclebits(tag::Int, n1::Int,n2::Int, N1::Int, N2::Int)
    #=Performs a cyclic permutations of n1 steps to the right and n2 steps upward.
       Inputs: tag: tag of the reference state
               n1: cycle times of n1-direction, columns,if n1=0, no cycle in columns
               n2: cycle times of n2-direction, rows,if n2=0, no cycle in rows
               N1: number of columns
               N2: number of rows
       Outputs: ntag: the tag of the outcoming state=#
    SiteNum = N1 * N2 * 2
    a = reshape(1: 2: SiteNum, N1, N2)
    b = reshape(2: 2: SiteNum, N1, N2)
    a = circshift(a, (n1, n2))
    b = circshift(b, (n1, n2))
    #ncfig = reshape(b, (SiteNum, 1))
    #println(ncfig)
    ntag = 0
    for i = 1: (N1*N2)
        ntag += 2^(SiteNum - 2*i + 1 ) * bits(tag, a[i], N1, N2)
        ntag += 2^(SiteNum - 2*i) * bits(tag, b[i], N1, N2)
    end  
    return ntag
end

function MomBasis(k1::Int, k2::Int, N1::Int, N2::Int)
    #=Momentum basis list of a given k1, k2
       Inputs: k1: momentum in n1-direction
               k2: momentum in n2-direction
               N1: number of columns
               N2: number of rows
       Outpus: mbasis: momentum basis list
               Ra: list of corresponding cycle period 
                   R1: n1-direction 
                   R2: n2-direction=#
    dim = 2^(N1 * N2 * 2)
    
    mask1 = - ones(Int, dim)
    mask2 = - ones(Int, dim)
    basis = [x for x = 1: dim]
        
    #println("Finish initialize")
    # Translation symmetry along n1-direction
    for tag = 0: (dim - 1)
        if mask1[tag + 1] == -1
            for n1 = 1: N1
                ntag = cyclebits(tag, n1, 0, N1, N2)
                if ntag > tag
                    mask1[ntag + 1] = 0
                elseif ntag == tag
                    if k1%(N1/n1) == 0
                        mask1[ntag + 1] = n1
                    else
                        mask1[ntag + 1] = 0
                    end
                    #println(m1)
                    break
                elseif ntag < tag
                    if mask1[ntag + 1] < 0
                        println("missing one element at n1!")
                    end
                end
            end
        end
    end
    
    
    # Translation symmetry along n2-direction
    for tag = 0: (dim - 1)
        if mask2[tag + 1] == -1
            for n2 = 1: N2
                ntag = cyclebits(tag, 0, n2, N1, N2)
                if ntag > tag
                    mask2[ntag + 1] = 0
                elseif ntag == tag
                    if k2%(N2/n2) == 0
                        mask2[ntag + 1] = n2
                    else
                        mask2[ntag + 1] = 0
                    end
                    #println(m1)
                    break
                elseif ntag < tag
                    if mask2[ntag + 1] < 0
                        println("missing one element at n2!")
                    end
                end
            end
        end
    end
    
    for i = 1:dim
        mask1[i] = mask1[i] * mask2[i]
        basis[i] = basis[i] * sign(mask1[i])
    end
    
    ra = filter!(x->x!=0, mask1)
    mb = filter!(x->x!=0, basis)
    mb = replace!(x -> x-1, mb)
    return mb, ra
end

function MomBasis_opt(k1::Int, k2::Int, N1::Int, N2::Int)
    #=Momentum basis list of a given k1, k2
       Inputs: k1: momentum in n1-direction
               k2: momentum in n2-direction
               N1: number of columns
               N2: number of rows
       Outpus: mbasis: momentum basis list
               Ra: list of corresponding cycle period 
                   R1: n1-direction 
                   R2: n2-direction=#
    # Generate all information about the basis
    dim = 2^(N1 * N2 * 2)
    
    mask1 = - ones(Int, dim)
    mask2 = - ones(Int, dim)
    basis = [x for x = 1: dim]
    
    t1 = spzeros(Int, dim, dim)
    t2 = spzeros(Int, dim, dim)
        
    #println("Finish initialize")
    # Translation symmetry along n1-direction
    for tag = 0: (dim - 1)
        if mask1[tag + 1] == -1
            for n1 = 1: N1
                ntag = cyclebits(tag, n1, 0, N1, N2)
                if ntag > tag
                    mask1[ntag + 1] = 0
                    t1[tag + 1, ntag + 1] = n1
                elseif ntag < tag
                    if mask1[ntag + 1] < 0
                        println("missing one element at n1!")
                    end
                elseif ntag == tag
                    t1[tag + 1, ntag + 1] = n1
                    if k1%(N1/n1) == 0
                        mask1[ntag + 1] = n1
                    else
                        mask1[ntag + 1] = 0
                    end
                    
                    break

                end
            end
        end
    end
    
    
    # Translation symmetry along n2-direction
    for tag = 0: (dim - 1)
        if mask2[tag + 1] == -1
            for n2 = 1: N2
                ntag = cyclebits(tag, 0, n2, N1, N2)
                if ntag > tag
                    mask2[ntag + 1] = 0
                    t2[tag + 1, ntag + 1] = n2
                elseif ntag < tag
                    if mask2[ntag + 1] < 0
                        println("missing one element at n2!")
                    end
                elseif ntag == tag
                    t2[tag + 1, ntag + 1] = n2
                    if k2%(N2/n2) == 0
                        mask2[ntag + 1] = n2
                    else
                        mask2[ntag + 1] = 0
                    end
                    #println(m1)
                    break

                end
            end
        end
    end
    
    for i = 1:dim
        mask1[i] = mask1[i] * mask2[i]
        basis[i] = basis[i] * sign(mask1[i])
    end
    
    ra = filter!(x->x!=0, mask1)
    mb = filter!(x->x!=0, basis)
    mb = replace!(x -> x-1, mb)
    return mb, ra, t1, t2
end

MomBasis_opt (generic function with 1 method)

In [179]:
N1=1
N2=2
k1=0
k2=0
mb, ra, t1, t2 = MomBasis_opt(k1, k2, N1, N2)
println(mb)

[0, 1, 2, 3, 5, 6, 7, 10, 11, 15]


In [40]:
@benchmark mb, ra = MomBasis(k1, k2, N1, N2)

BenchmarkTools.Trial: 
  memory estimate:  2.16 MiB
  allocs estimate:  16395
  --------------
  minimum time:     3.014 ms (0.00% GC)
  median time:      3.053 ms (0.00% GC)
  mean time:        3.160 ms (2.72% GC)
  maximum time:     40.867 ms (91.22% GC)
  --------------
  samples:          1582
  evals/sample:     1

\section{Hamiltonian matrix}

In [270]:
# Hamiltonian matrix
function minstate(tag::Int, t::SparseMatrixCSC{Int64,Int64})
    dim = size(t,1)
    ms = 0
    for i = 1: dim
        if t[i, tag + 1] != 0
            ms = i - 1
            break
        end
    end
    return ms
end

function findstate(tag::Int, mb::Array{Int,1}, t1::SparseMatrixCSC{Int64,Int64}, t2::SparseMatrixCSC{Int64,Int64}, N1::Int, N2::Int)
    #=Find the reference state of an arbitrary state.
       Inputs: tag :tag of the input state
               mb: list of momentum basis
               t1, t2: Translation matrix
       Outputs: ntag: tag of the corresponding reference state
                l : translation steps
                ntag and l will return negative values if tag is not in mbasis.=#     
    l1 = -1
    l2 = -1
    
    ms1 = minstate(tag, t1)
    l1 =  N1 - t1[ms1 + 1, tag + 1]
    if ms1 in mb
        l2 = 0
        return ms1, l1, l2
    else
        ms2 = minstate(ms1, t2)
        if ms2 in mb
            l2 = N2 - t2[ms2 + 1, ms1 + 1]
            return ms2, l1, l2
        else
            ms2 = -1
            l2 = -1
            l1 = -1
            return ms2, l1, l2
        end
    end
end

function findindex(tag::Int, mb::Array{Int,1})
    # find the index of a tag in mb
    # binary search
    dim = length(mb)
    low = 1
    high = dim   
    while low <= high
        mid = low + div(high - low, 2)
        if tag < mb[mid]
            high = mid -1
        elseif tag > mb[mid]
            low = mid + 1
        elseif tag == mb[mid]
            return mid
        end
    end
end
                           
function KitaevH(J::Array{Float64,1}, N1::Int, N2::Int, k1::Int, k2::Int)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice for a given k (with translation symmetry)
       N1: number of unit cells in n1-direction, column number
       N2: number of unit cells in n1-direction, row number
    
       Inputes: 
           N1, N2: width and length of the rhomboid cluster
           k1, k2: momentund
        
       Outputs: The Non-zero Hamiltonian element H[i] is in the 
                position (row[i], col[i])
                H, row, col : type: list
        =#
    mb, ra = MomBasis(k1, k2, N1, N2)
   
    dim = length(mb)
    f1 = -1im * 2 * pi / N1
    f2 = -1im * 2 * pi / N2
    
    base_dic = Dict{Int,Int}()
    for i = 1: dim
        base_dic[mb[i]] = i
    end
    
    data = Complex[]
    row = Int[]
    col = Int[]
    
    for tag in mb
        ind  = base_dic[tag]
        temp = Dict{Int, Complex}() # A temporay dict
        
        for r = 0: N2-1
            for c = 0: N1-1
                ini = index(N1, N2, c, r, 0)
                
                # Sx-Sx
                next = index(N1, N2, c, r, 1)
                
                ntag = flip(tag, ini, next, N1, N2)
                rtag, l1, l2 = findstate(ntag, mb, N1, N2)
                if rtag > 0
                    nind = base_dic[rtag]
                    phase = exp(f1*k1*l1 + f2*k2*2) * sqrt(ra[ind]/ra[nind])
                    if nind in keys(temp)
                        temp[nind] += -J[1]/4 * phase
                    else
                        temp[nind] = -J[1]/4 * phase
                    end
                end
                                
                # Sy-Sy
                next = index(N1, N2, c - 1, r, 1)
                    
                ntag = flip(tag, ini, next, N1, N2)
                rtag, l = findstate(ntag, mb, N1, N2)
    
                if rtag > 0
                    nind = base_dic[rtag]
                    phase = exp(f1*k1*l1 + f2*k2*l2) * sqrt(ra[ind]/ra[nind])
                    
                    if nind in keys(temp)
                        temp[nind] += J[2]*(bits(tag, ini, N1, N2)-0.5)*(bits(tag, next, N1, N2)-0.5)*phase           
                    else
                        temp[nind] = J[2]*(bits(tag, ini, N1, N2)-0.5)*(bits(tag, next, N1, N2)-0.5)*phase
                    end
                end
                                    
                # Sz-Sz
                next = index(N1, N2, c, r - 1, 1)
                    
                if ind in keys(temp)
                    temp[ind] += - J[3] * (bits(tag, ini, N1, N2)-0.5)*(bits(tag, next, N1, N2)-0.5)
                else
                    temp[ind] = - J[3] *(bits(tag, ini, N1, N2)-0.5)*(bits(tag, next, N1, N2)-0.5)
                end
            end
        end
                        
        #odTemp = sort!(collect(keys(temp)))
        for sf in keys(temp)
            append!(col,[ind])
            append!(row,[sf])
            append!(data,[temp[sf]])
        end
    end
    return col, row, data
end

function KitaevH_opt(J::Array{Float64,1}, N1::Int, N2::Int, k1::Int, k2::Int)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice for a given k (with translation symmetry)
       N1: number of unit cells in n1-direction, column number
       N2: number of unit cells in n1-direction, row number
    
       Inputes: 
           N1, N2: width and length of the rhomboid cluster
           k1, k2: momentund
        
       Outputs: The Non-zero Hamiltonian element H[i] is in the 
                position (row[i], col[i])
                H, row, col : type: list
        =#
    mb, ra, t1, t2 = MomBasis_opt(k1, k2, N1, N2)
    
    dim = length(mb)
    f1 = -1im * 2 * pi / N1
    f2 = -1im * 2 * pi / N2
    
    H = spzeros(Complex, dim, dim)

    for tag in mb
        i = findindex(tag, mb)
        for r = 0: N2-1
            for c = 0: N1-1
                si = index(N1, N2, c, r, 0) # start index
                               
                # Sx-Sx
                ni = index(N1, N2, c, r, 1) # next index
                ntag = flip(tag, si, ni, N1, N2)
                rtag, l1, l2 = findstate(ntag, mb, t1, t2, N1, N2)
                if rtag > 0
                    j = findindex(rtag, mb)
                    phase = exp(f1*k1*l1 + f2*k2*l2) * sqrt(ra[i]/ra[j])
                    H[j, i] += -J[1]/4 * phase
                end
                               
                # Sy-Sy
                ni = index(N1, N2, c - 1, r, 1)
                ntag = flip(tag, si, ni, N1, N2)
                rtag, l1, l2 = findstate(ntag, mb, t1, t2, N1, N2)
                if rtag > 0
                    j = findindex(rtag, mb)
                    phase = exp(f1*k1*l1 + f2*k2*l2) * sqrt(ra[i]/ra[j])
                    H[j, i] += J[2]*(bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5)*phase
                end
                                    
                # Sz-Sz
                ni = index(N1, N2, c, r - 1, 1)
                H[i, i] += - J[3] * (bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5)
            end
        end
    end
    return H
end

KitaevH_opt (generic function with 1 method)

In [271]:
using SparseArrays
using Arpack

N1 = 2
N2 = 3
SiteNum = N1 * N2 * 2

J = -[1.0,1.0,1.0]
for k1 = 0: N1-1
    for k2 = 0: N2-1
        H = KitaevH_opt(J, N1, N2, k1, k2)
        e, x = eigs(H, which = :SR)
        println(real(e/SiteNum))
    end
end

[-0.203346, -0.202673, -0.19379, -0.191417, -0.19109, -0.189255]
[-0.202673, -0.193735, -0.19109, -0.189255, -0.187684, -0.185959]
[-0.202673, -0.193735, -0.19109, -0.189255, -0.187684, -0.185959]
[-0.191978, -0.188293, -0.187178, -0.186273, -0.18626, -0.182731]
[-0.191978, -0.188293, -0.187178, -0.186273, -0.182731, -0.182693]
[-0.191978, -0.188293, -0.187178, -0.186273, -0.182731, -0.182693]
